<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/PySyft_FashionMNIST-Federated-Learning_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install syft

In [0]:
import time
import datetime
import os
import sys

In [0]:
!pip install tf-encrypted
! URL="https://github.com/Berenice2018/PySyft-Bc.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b master --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft-Bc; python setup.py install

module_path = os.path.abspath(os.path.join('./PySyft-Bc'))
if module_path not in sys.path:
     sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import syft as sy

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory




W0725 13:27:06.922409 140489880336256 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0725 13:27:06.943093 140489880336256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# create workers, 
hook = sy.TorchHook(torch)

ada = sy.VirtualWorker(hook, 'ada')
bob = sy.VirtualWorker(hook, 'bob')
cyd = sy.VirtualWorker(hook, 'cyd')

In [0]:
def create_loaders():
    print('creating loaders')
    # define the transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))
    ])

    # load the datasets
    fulltrainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
    testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)

    train_size = int(len(fulltrainset)* 0.8)
    valid_size = len(fulltrainset) - train_size

    # split the dataset
    #trainset, validationset = torch.utils.data.random_split(testset, [train_size, valid_size])
    #trainset = trainset.dataset
    #validationset = validationset.dataset
    
    # federate
    federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
      datasets.FashionMNIST('.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5,), (0.5,))
                   ]))
    .federate((ada, bob, cyd)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=32, shuffle=True)

    #federated_valid_loader = sy.FederatedDataLoader(
    #    validationset.federate((ada,bob,cyd)), batch_size=32, shuffle=True)

    valid_loader = torch.utils.data.DataLoader(testset, batch_size=32)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)
    
    # check that our trainloader returns a pointer to a batch, and that transformations are applied
    #data, labels = next(iter(federated_train_loader))
    #print(data)
    
    return federated_train_loader, valid_loader, test_loader

**Architecture**

In [0]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    



**Train and validate**

In [0]:


# Create the data loaders, federated PySyft loaders are returned
fed_train_loader, fed_valid_loader, test_loader = create_loaders()

len_train_loader = len(fed_train_loader)
len_valid_loader = len(fed_valid_loader)
len_train_loader


creating loaders


1875

In [0]:
# Helper functions for printing oput training progress data
def print_epoch_start_stats(e_start, e_end, current_lr, current_vmin):

    print('*** Epoch [{}/{}]: Training with LR [{:.6f}], current VLoss Min [{:.4f}]'.format(
    e_start, e_end, current_lr, current_vmin))

def print_epoch_end_stats(train_loss, valid_loss, valid_acc, epoch_time):

    print('   Train loss: \t{:.6f}'.format(train_loss))
    print('   Valid loss: \t{:.6f}'.format(valid_loss))
    print('   Valid acc: \t{:.6f}'.format(valid_acc))
    print('*** Epoch completed in {:.0f}m {:.0f}s'.format(epoch_time // 60, epoch_time % 60))   

In [0]:


print(f'objects of ada= {len(ada._objects)}, bob= {len(bob._objects)}, cyd= {len(cyd._objects)}')


objects of ada= 4, bob= 4, cyd= 4


In [0]:
# helper functions
import datetime

def get_time():
      hour = datetime.datetime.today().hour +2
      minute = datetime.datetime.today().minute
      second = datetime.datetime.today().second
      return hour, minute, second

def train_epoch(model, dataloader, criterion, optimizer, train_on_gpu=False):
    # initialize variables to monitor training and validation loss
    train_loss = 0.0
    train_accuracy = 0.0
    correct = 0.0
    total = 0.0
    
    for batch_idx, (data, target) in enumerate(dataloader):
        # move to GPU
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        model.send(data.location) # send the model to the right location
        
        ## find the loss and update the model parameters accordingly
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        model.get() # get the model back
        current_loss = loss.get()
        
        # get the loss per batch and accumulate
        train_loss += current_loss.item()
        
        # get the class, highest probability
        probabilities = torch.exp(output)
        _, top_class = probabilities.topk(1, dim=1)
        
        # check if the predicted class is correct
        equals = top_class == target.view(*top_class.shape)
        
        #train_accuracy += torch.mean(equals.type(torch.FloatTensor))
        train_accuracy += torch.mean(torch.tensor(equals))

    return train_loss, train_accuracy


def validate_epoch(model, dataloader, criterion, train_on_gpu=False):
    valid_loss = 0.0
    valid_accuracy = 0.0
    correct = 0.0
    total = 0.0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            # move to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output,target)
            
            valid_loss += loss.item()

            ps = torch.exp(output)
            _ , top_class = ps.topk(1,dim = 1)
            #_, top_class = torch.max(ps, dim=1)
            equals = top_class == target.view(*top_class.shape) # shape is (batch size x 1)
            valid_accuracy += torch.mean(equals.type(torch.FloatTensor))
            
            #pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            #correct += pred.eq(target.view_as(pred)).sum().item()

    return valid_loss, valid_accuracy

In [0]:
def train_my_model(n_epochs, loaders, model, optimizer, criterion, scheduler, use_cuda=False):
    print('Training started at ', get_time())
    
    valid_losses = []
    train_losses = []
    valid_accuracies = []
    
    
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(n_epochs):
        
         # initialize variables to monitor training and validation loss
        training_loss = 0.0
        training_accuracy = 0.0
    
        #if scheduler is not None:
          #scheduler.step()
        
        ###################
        # train the model #
        model.train()
        training_loss, training_accuracy = train_epoch(model, loaders[0], criterion, optimizer, use_cuda)
    
        
        ######################    
        # validate the model #
        model.eval()
        validation_loss = validate_epoch(model, loaders[1], criterion, use_cuda) #validation_accuracy
        
        #if scheduler is not None:
          #scheduler.step(validation_loss)
        
        ###### print training/validation statistics 
        # calculate the average loss per epoch
        training_loss = training_loss/len_train_loader
        train_losses.append(training_loss)
        
        training_accuracy = training_accuracy/len_train_loader
        
        validation_loss = validation_loss/len_valid_loader
        valid_losses.append(validation_loss)
        
        validation_accuracy = validation_accuracy/len_valid_loader
        valid_accuracies.append(validation_accuracy)
        
        hour, minute, second = get_time()
        print('Epoch: {} at {}:{}:{} \tTrain. Loss: {:.6f} \tValid. Loss: {:.6f} \t Accur.: {:.10f}'.format(
                  epoch,
                  hour, minute, second,
                  training_loss,
                  #training_accuracy, 
                  validation_loss,
                  validation_accuracy ))
        
        ###### TODO: save the model if validation loss has decreased
        if validation_loss <= valid_loss_min:
            '''print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                validation_loss))'''
            print('Validation loss decreased by {:.6f}'.format(validation_loss - valid_loss_min))
            #torch.save(model.state_dict(), save_path)
            valid_loss_min = validation_loss
            
            
    ##### visualize
    #plot_loss_acc(n_epochs, train_losses, valid_losses, valid_accuracies)
    
    return model

In [0]:
# instantiate the model
model = Model()


class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 2
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 4)
criterion = nn.NLLLoss()


In [0]:
######## 
# start the training
###########


loaders = [fed_train_loader, fed_valid_loader, test_loader]
n_epochs = 2
train_my_model(n_epochs, loaders, model, optimizer, criterion, scheduler)



In [0]:
ada.clear_objects()
bob.clear_objects()
cyd.clear_objects()

<VirtualWorker id:cyd #objects:0>